In [1]:
from jina import Flow, Executor, requests, Document, DocumentArray
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Games.csv')
df= df.drop_duplicates().dropna()

In [3]:
df.head()

,Console,GameName,Review,Score
0,PC,Baldur's Gate 3 Early Access Review,Early Access,6
1,NS,Control: Ultimate Edition Cloud Version Review,Good,7
2,"XONE, PC, PS4",Doom Eternal: The Ancient Gods Part 1 Review,Good,7
3,"XONE, PC, PS4",Watch Dogs: Legion Review,Great,8
4,PC,Ring Of Pain Review,Great,8


In [4]:
Document(text ="text")
Document(content ="content")
Document(uri = "path" )


In [5]:
docs = DocumentArray()
for ind in range(df.shape[0]):
    name = df.iloc[ind,1]
    desc = df.iloc[ind,2]
    rating = df.iloc[ind,3]
    # print(name,"-",desc,"\n")
    doc = Document(text=name)
    doc.tags['Review'] = desc
    docs.append(doc)

In [6]:
flow = (
    Flow()
    .add(uses='jinahub+docker://SpacyTextEncoder')
    .add(uses='jinahub+docker://SimpleIndexer')
    .add().plot('f.svg')
)

In [ ]:
with flow:
        flow.index(inputs=docs)

Output()

Output()

           Flow@17588[I]:🎉 Flow is ready to use!                                          
	🔗 Protocol: 		GRPC
	🏠 Local access:	127.0.0.1:51099
	🔒 Private network:	192.168.18.37:51099


In [ ]:
text=input("Game Name:")
with flow:
    response = flow.search(inputs=text, return_results=True)

In [ ]:
matches = response[0].data.docs[0].matches

print("Your search results")
print("-------------------\n")
i=0

In [ ]:
for match in matches:
    i+=1
    if(i==1):
        print("Closest Match for",match.text,"Review : ", match.tags.fields['Review'].string_value,"\n")
    else:
        print("Similar Query",i)
        print("-------------------\n")
        print(match.tags.fields['Review'].string_value)

In [4]:
for ind in range(5):
    print()
    print(f" Game name : {df.iloc[ind, 1]} ".center(80, ' '))
    print()
    print(f" Console : {df.iloc[ind, 0]} ")
    print()
    print(f" Review  : {df.iloc[ind, 2]} ")
    print()
    print(f" Score   : {df.iloc[ind, -1]} ")
    print()
    print('-' * 80)


                Game name : Baldur's Gate 3 Early Access Review                 

 Console : PC 

 Review  : Early Access 

 Score   : 6 

--------------------------------------------------------------------------------

           Game name : Control: Ultimate Edition Cloud Version Review           

 Console : NS 

 Review  : Good 

 Score   : 7 

--------------------------------------------------------------------------------

            Game name : Doom Eternal: The Ancient Gods Part 1 Review            

 Console : XONE, PC, PS4 

 Review  : Good 

 Score   : 7 

--------------------------------------------------------------------------------

                     Game name : Watch Dogs: Legion Review                      

 Console : XONE, PC, PS4 

 Review  : Great 

 Score   : 8 

--------------------------------------------------------------------------------

                        Game name : Ring Of Pain Review                         

 Console : PC 

 Review  : Great 


In [6]:
docs = DocumentArray()
for ind in range(df.shape[0]):
    name = df.iloc[ind, 1]
    score = df.iloc[ind,-1]
    doc = Document(text = name)
    docs.append(doc)
    
docs

<jina.types.arrays.document.DocumentArray length=14785 at 2313409629536>

In [8]:
docs[1].json()

'{\n  "id": "f9c2efef-3e21-11ec-a622-a4b1c1874569",\n  "mime_type": "text/plain",\n  "text": "Control: Ultimate Edition Cloud Version Review"\n}'

In [9]:
class Clean(Executor):
    @requests
    def clean(self, docs, parameters, **kwargs):
        print(parameters)
        return DocumentArray(list(filter(lambda x : len(x.text) > 0, docs)))

In [10]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1"

flow = (
    Flow()
    .add(
        name = 'clean_text',
        uses = Clean
    )
    .add(
        name="error_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="error_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

In [11]:
docs = docs[:1000]
docs

<jina.types.arrays.document.DocumentArray length=1000 at 2313409626800>

In [14]:
with flow:
    flow.index(
      inputs=docs,
        docs = docs,
        parameters = {'name' : 'somethign', 'xyz' : 'fsdfsdfsa'}
  )

Output()

     clean_text@18692[W]:Pea is being closed before being ready. Most likely some other Pea in the Flow or Pod failed to start
     clean_text@18692[W]:Terminating process after waiting for readiness signal for graceful shutdown


InvalidRequirement: Parse error at "'-f https'": Expected W:(abcd...)

In [ ]:
query = Document(text = input('Query product : '))
with flow:
    response = flow.search(inputs = query, return_results = True)

In [ ]:
matches = response[0].docs[0].matches
matches

In [ ]:
for m in matches:
    print()
    print(f" product name : {m.text} ".center(80, ' '))
    print()
    print(m.tags['description'])
    print()
    print('-' * 80)